In [148]:
import numpy as np
import pandas as pd
import do_mpc
from casadi import *
import sys

In [157]:
# Given tangential velocity and radius, calculate angular velocity in degrees per second

tangential_velocity = 5  # m/s
radius = 1  # meters

# Angular velocity in radians per second
angular_velocity_rad = tangential_velocity / radius

# Convert to degrees per second
angular_velocity_deg = angular_velocity_rad * (180 / np.pi)  # radians to degrees conversion
angular_velocity_deg


286.4788975654116

In [162]:
angular_velocity_deg = 360

radius = 0.075

angular_velocity_rad = angular_velocity_deg * (np.pi / 180)  # degrees to radians conversion

tangential_velocity = angular_velocity_rad * radius

tangential_velocity # m/s


0.47123889803846897

In [150]:
# MODEL DEFINITION

model_type = 'continuous'
model = do_mpc.model.Model(model_type)

# Define some system constraints
yaw_theta_max = 45  # degrees
yaw_theta_min = -45  # degrees
yaw_omega_max = 100 * 360 / 60  # 100 RPM to deg/s

# Assuming theta is the angular position of the servo and omega is its angular velocity
theta = model.set_variable('_x', 'theta')
omega = model.set_variable('_x', 'omega')

# Control input for the servo (angular velocity command)
control_omega = model.set_variable('_u', 'control_omega')

# Target's position and velocity as external inputs
target_theta = model.set_variable('_tvp', 'target_theta')
target_omega = model.set_variable('_u', 'target_omega')

# Incorporate uncertainty parameters into your system's equations
alpha = model.set_variable('_p', 'alpha')
beta = model.set_variable('_p', 'beta')

# System equations
model.set_rhs('theta', omega)  # Change in angle is current angular velocity
model.set_rhs('omega', control_omega)  # Control input directly sets angular velocity

model.setup()


In [151]:
# CONTROLLER DEFINITION

mpc = do_mpc.controller.MPC(model)

setup_mpc = {
    'n_horizon': 20,
    'n_robust': 1,
    'open_loop': 0,
    't_step': 0.005,
    'state_discretization': 'collocation',
    'collocation_type': 'radau',
    'collocation_deg': 2,
    'collocation_ni': 2,
    'store_full_solution': True,
    # Use MA27 linear solver in ipopt for faster calculations:
    #'nlpsol_opts': {'ipopt.linear_solver': 'MA27'}
}

mpc.set_param(**setup_mpc)

# Apply constraints to the model
mpc.bounds['lower','_x', 'theta'] = yaw_theta_min
mpc.bounds['upper','_x', 'theta'] = yaw_theta_max
mpc.bounds['lower','_u', 'control_omega'] = -yaw_omega_max
mpc.bounds['upper','_u', 'control_omega'] = yaw_omega_max

# OBJECTIVE FUNCTION DEFINITION

# Weights for the cost function components
w_theta = 1.0  # Weight for position error
w_omega = 0.4  # Weight for velocity error

# Define the stage cost (cost at each time step in the horizon)
lterm = w_theta * (model.x['theta'] - model.tvp['target_theta'])**2 + w_omega * (model.x['omega'] - model.u['target_omega'])**2

# Define the terminal cost (cost at the end of the prediction horizon)
mterm = w_theta * (model.x['theta'] - model.tvp['target_theta'])**2  # You can modify this as needed

mpc.set_objective(lterm=lterm, mterm=mterm)

# You can also include a penalty on the change in control input if desired
mpc.set_rterm(control_omega=0.01)  # This is a small penalty for large changes in control input

alpha_var = np.array([1., 1.05, 0.95])
beta_var = np.array([1., 1.1, 0.9])

mpc.set_uncertainty_values(alpha = alpha_var, beta = beta_var)

In [152]:

# ESTIMATOR DEFINITION

# assume all states can be directly measured ie state feedback
estimator = do_mpc.estimator.StateFeedback(model)

# SIMULATOR DEFINITION

initial_theta = 5
initial_omega = 0

# Create closed loop simulator to run MPC
simulator = do_mpc.simulator.Simulator(model)

params_simulator = {
    'integration_tool': 'cvodes',
    'abstol': 1e-8,
    'reltol': 1e-8,
    't_step': 0.005,
}

simulator.set_param(**params_simulator)

p_num = simulator.get_p_template()
tvp_num = simulator.get_tvp_template()

# function for time-varying parameters
def tvp_fun(t_now):
    return tvp_num

# uncertain parameters
p_num['alpha'] = 1
p_num['beta'] = 1

def p_fun(t_now):
    return p_num

simulator.set_tvp_fun(tvp_fun)

# Set the TVP function in the MPC
mpc.set_tvp_fun(tvp_fun)

simulator.set_p_fun(p_fun)

# Define the simulator's initial state
x0 = np.array([[initial_theta], [initial_omega]])
simulator.x0 = x0
mpc.x0 = x0

mpc.setup()
simulator.setup()

AssertionError: Incorrect output of tvp_fun. Use get_tvp_template to obtain the required structure.

In [ ]:
# Prepare for storing simulation data
history = {
    'theta': [],
    'omega': [],
    'control_omega': []
}

# Simulation loop
for k in range(100):  # Let's assume 100 time steps for this example
    u0 = mpc.make_step(x0)
    y_next = simulator.make_step(u0)
    
    # Store the simulation data
    history['theta'].append(y_next['theta'])
    history['omega'].append(y_next['omega'])
    history['control_omega'].append(u0)

    # Update the state
    x0 = y_next